In [6]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import torch
from data.diw import DIW, DIWDataAug
from torch.utils.data import DataLoader
from data.utils import make_transforms
from networks.paperedge import GlobalWarper, LocalWarper
import os
import glob
import re
import torchvision.transforms as transforms
import numpy as np

In [2]:
root_dir = '/home/gem/Downloads/unwrarp/dataset/selected_real_dataset'
diw_loader = DataLoader(DIW(root_dir, transforms=make_transforms()), num_workers=4, batch_size=2, shuffle=True, pin_memory=True)
# diw_dataset = DIW(root_dir, transforms=make_transforms())
diw_aug = DIWDataAug()

In [1]:
import torch

/home/gem/anaconda3/envs/paper_edge/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
netG = GlobalWarper()
netG.load_state_dict(state_dict=model_dict['G'])

/home/gem/anaconda3/envs/paper_edge/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


<All keys matched successfully>

In [7]:
# im, ms, bg = diw_dataset[0]

In [9]:
batch[1].shape

torch.Size([2, 1, 512, 512])

In [7]:
len(batch[0].nonzero(as_tuple=True))

4

In [3]:
batch = next(iter(diw_loader))
ims, msks = diw_aug(*batch)

In [21]:
im_path = './data/5k/img/00Nt5b_4.png'
fmp = './data/5k/seg/00Nt5b_4.png'
im = cv2.imread(im_path)
fm = cv2.imread(fmp)
fm[fm > 0] = 255
gray_im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

In [32]:
fm_tensor = torch.tensor(fm.mean(keepdims=True, axis=2).transpose(2, 0, 1))
x = fm_tensor.nonzero(as_tuple=True)

In [47]:
ret, thresh = cv2.threshold(gray_im, 100, 255, cv2.THRESH_BINARY)

In [48]:
# # detect the contours on the binary image using cv2.CHAIN_APPROX_NONE
# contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
                                      
# # draw contours on the original image
# image_copy = im.copy()
# im_with_contour = cv2.drawContours(image=image_copy, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

In [2]:
from transformers import MaskFormerImageProcessor, MaskFormerForInstanceSegmentation

processor = MaskFormerImageProcessor.from_pretrained("facebook/maskformer-swin-large-ade")
model = MaskFormerForInstanceSegmentation.from_pretrained("facebook/maskformer-swin-large-ade")

/home/gem/anaconda3/envs/paper_edge/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/gem/anaconda3/envs/paper_edge/lib/python3.8/site-packages/transformers/models/maskformer/image_processing_maskformer.py:401: FutureWarning: The `size_divisibility` argument is deprecated and will be removed in v4.27. Please use `size_divisor` instead.
  warnings.warn(
/home/gem/anaconda3/envs/paper_edge/lib/python3.8/site-packages/transformers/models/maskformer/image_processing_maskformer.py:408: FutureWarning: The `max_size` argument is deprecated and will be removed in v4.27. Please use size['longest_edge'] instead.
  warnings.warn(


In [3]:
from PIL import Image

im_path = "./data/real-dataset/控除証明書データ/ダウンロード (2).jpg"
image = Image.open(im_path)
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
# model predicts class_queries_logits of shape `(batch_size, num_queries)`
# and masks_queries_logits of shape `(batch_size, num_queries, height, width)`
class_queries_logits = outputs.class_queries_logits
masks_queries_logits = outputs.masks_queries_logits

# you can pass them to processor for postprocessing
# we refer to the demo notebooks for visualization (see "Resources" section in the MaskFormer docs)
predicted_semantic_map = processor.post_process_semantic_segmentation(outputs, target_sizes=[image.size[::-1]])[0]
